In this example we will convert the simple SFINCS compound flood model that was made in the notebook 'build_simple_compound_model_from_script.ipynb' to a subgrid type SFINCS model, using the underlying Python functions of HydroMT-SFINCS to build the model.

The model is situated in Northern Italy, where a small selection of topography and bathymetry data has already been made available for you to try the examples.

In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import geopandas as gpd

from hydromt_sfincs import SfincsModel
from hydromt_sfincs import utils

This example shows how to turn a simple SFINCS model on a regular grid, to one with subgrid-derived tables.
It also still contains offshore water level forcing and an upstream discharge input forcing.

For making a more advanced model including e.g. spatially varying infiltration and roughness, see the example notebook: build_advanced_subgrid_compound_model_from_script.ipynb

In case you want to adjust this example to build a SFINCS model anywhere else in the world, you will have to add your own datasets to HydroMT's data catalog. For more info on that, see the example notebook: example_datasources.ipynb

Steps followed in this notebook to build your SFINCS model:
<ul> 
<li> 1. Open SfincsModel class, set data library and output folder to read from </li>
<li> 2. Load in existing regular SFINCS model </li>
<li> 3. Load in wanted elevation datasets </li>
<li> 4. Rebuild model with subgrid derived tables </li>
<li> 5. Show model</li>
<li> 6. Save all files</li>
</ul> 

Let's get started!

1. Open SfincsModel class, set data library and output folder to read from:

In [ ]:
# Initialize SfincsModel Python class with the artifact data catalog which contains publically available data for North Italy
# Also read in the regular grid SFINCS model that was already made in the folder sfincs_compound:
sf = SfincsModel(data_libs=["artifact_data"], root="sfincs_compound", mode="r+") 


2. Load in existing regular SFINCS model:

In [ ]:
# Reading in the existing model is as easy as this:
sf.read()

In [ ]:
# The input file was automatically read in. See what it looks like displayed below:
sf.config

In [ ]:
# You can for instance inspect the grid:
sf.grid

In [ ]:
# Or the model forcing:
sf.forcing

In [ ]:
# Or the spatial attributes (=geoms) of e.g. the region or observation points
sf.geoms

3. Load in wanted elevation datasets:

You need to specify again what elevation datasets you want to use to build your subgrid derived tables, these could in principle be different from before, here we keep them the same.

In [ ]:
# In this example we want to combine 2 elevation datasets, merit_hydro as elevation and gebco as bathymetry, in that order. 

# Here these 2 are loaded from the datacatalog and added to one list of elevation datasets (you could add more yourself if wanted, in case made available in your data catalog)
da_dep1 = sf.data_catalog.get_rasterdataset("merit_hydro", variables=["elevtn"], geom=sf.region, buffer=5)

da_dep2 = sf.data_catalog.get_rasterdataset("gebco", variables=["elevtn"], geom=sf.region, buffer=5)

# Provide merge arguments together with xr.DataAraay
da_dep_lst = [{"da":da_dep1}, {"da":da_dep2, "zmax":0, "offset":0}]

#NOTE: from the 2nd elevation dataset (gebco) only elevation data below an elevation of 0 meters is used ("zmax":0)

In [ ]:
# Again, make a plot of the merged topobathy, here colour limits are set between an elevation of -5 to 5 meters
sf.grid.dep.plot(x='xc', y='yc',vmin=-5, vmax=5)

4. Rebuild model with subgrid derived tables

In [ ]:
# NOTE: in case we don't want to overwrite the SFINCS model in the existing folder, we have to specify a new folder:
sf.set_root('sfincs_compound_subgrid')

Now we call the function to make the subgrid derived tables

In [ ]:
# Every single grid cell of the flux grid of the size inp.dx by inp.dy is defined into subgrid pixels (default is 20, nr_subgrid_pixels = 20).
# For every subgrid pixel the topobathy data is loaded, ideally this consists also of high-resolution DEM datasets that you specify as user.

# Manning roughness is still based on elevation in this example, above 'rgh_lev_land: 0.0' the valus is 'manning_land: 0.04',  below it is 'manning_sea: 0.02'
sf.create_subgrid(da_dep_lst=da_dep_lst, make_dep_tiles=True, make_manning_tiles=True)

# NOTE: we turned on that the merged topobathy and manning tiles of the different (high-res) datasets are written to tiff-files

# NOTE: if you have a very large domain with 100,000s to millions of cells, and very high-resolution datasets, this step might take minutes to hours!!!
#       But good news; when finished succesfully, you can very quickly run very accurate SFINCS simulations!
#       The whole point of the subgrid functionality of SFINCS is that by derived subgrid tables based on high res elevation data, 
#       you either have more accurate results or run on a coarser grid resolution (= much faster) or both

Now we can see what kind of subgrid-derived variables are created:

In [ ]:
sf.subgrid

4. Show model

You can inspect for instance the minimum found bed level per flux grid cell, based on the high-res subgrid pixels, called 'z_zmin':

In [ ]:
sf.subgrid.z_zmin.plot(x='xc', y='yc',vmin=-5, vmax=5)

5. Save all files

In [ ]:
sf.write() # write all

In [ ]:
# Show created files in folder:
dir_list = os.listdir(sf.root)
print(dir_list)

# NOTE: now there is no sfincs.dep file anymore, but a sfincs.sbg file that contains your subgrid-derived tables

Your basemap and forcing figures are saved in the folder 'figs', GIS files (tiff & geojson) of your model setup in 'gis' and intermediate merged subgrid elevation and manning roughness tiles in 'tiles'.

Now you have made a model, you can progress to the notebook: run_sfincs_model.ipynb